## Finetuning Chemberta for BBBP Property Prediction Task using LoRA Technique


In [1]:
#Importing Libraries

import evaluate
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from transformers import Trainer
from transformers import TrainingArguments, EarlyStoppingCallback
import pandas as pd
import numpy as np 

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the Datasets

In [2]:
import pandas as pd

train_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/train.csv')
val_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/valid.csv')

## Chemberta Models 

## Model 77M MLM model

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MLM",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 10M MLM model

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MLM")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MLM",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 10M MTR Model

In [ ]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-10M-MTR",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 77M MTR Model

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR",
    num_labels=5,
    trust_remote_code=True
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model 5M MTR Model

In [ ]:
## Model 5M MTR Model

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-5M-MTR")
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-5M-MTR",
    num_labels=5,
    trust_remote_code=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-5M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing Training and Validation Dataset

In [5]:
smiles_list= train_bbbp['smiles'].tolist()
smiles_val=val_bbbp['smiles'].tolist()
train_tokenized=tokenizer(smiles_list)
val_tokenized=tokenizer(smiles_val)

In [6]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_tokenized)
val_dataset = Dataset.from_dict(val_tokenized)

In [7]:
train_labels = train_bbbp['p_np'].tolist() # Assuming tasks start from column 1
val_labels = val_bbbp['p_np'].tolist()

In [8]:
train_dataset = train_dataset.add_column("labels", train_labels)
val_dataset = val_dataset.add_column("labels", val_labels)

### Compute Metrics


In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    probabilities= softmax(logits, axis=1)
    mcc = matthews_corrcoef(labels, predictions)

        
    return {
            "eval_mcc_metric": mcc,
            "Accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities,multi_class="ovr"),  # AUC-ROC requires probabilities
            "Precision": precision_score(labels, predictions,average="macro"),
            "Recall": recall_score(labels, predictions,average="macro"),
            "F1-score": f1_score(labels, predictions,average="macro")
        }

### define LoRA Config

In [ ]:

lora_config = LoraConfig(
    task_type="SEQ_CLS",  # Sequence classification task
    r=8,  # Rank of LoRA matrices
    lora_alpha=16,  # Scaling factor double of rank( from the rule of thumb)
    target_modules='all-linear',
    lora_dropout=0.1,  
    #init_lora_weights="gaussian"
)

model = get_peft_model(model, lora_config)

### Define Custom Trainer

In [ ]:
#focal loss computation

def focal_loss_multiclass(inputs, targets, alpha=1, gamma=2):
    log_prob = F.log_softmax(inputs, dim=-1)
    prob = torch.exp(log_prob)  # Convert log probabilities back to normal probabilities

    targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
    pt = torch.sum(prob * targets_one_hot, dim=-1)  # Get probability of the true class

    focal_loss = -alpha * (1 - pt) ** gamma * torch.sum(log_prob * targets_one_hot, dim=-1)
    
    return focal_loss.mean() 

In [ ]:

class CustomTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = focal_loss_multiclass(logits, labels)
        
        return (loss, outputs) if return_outputs else loss 

## Hyperparameter tuning using WandB

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [1]:
import wandb
import evaluate
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from evaluate import load
from datasets import Dataset
import numpy as np
import pandas as pd
import os
from scipy.special import softmax
from peft import LoraConfig, get_peft_model
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef

#Importing Libraries

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def data_load():
    train_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/train.csv')
    val_bbbp=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/valid.csv')

    return train_bbbp, val_bbbp

In [3]:
def data_prep(data_process,tokenizer):

    smiles_list = data_process['smiles'].tolist()
    tokenized=tokenizer(smiles_list)
    
    
    dataset = Dataset.from_dict(tokenized)
    

    labels = data_process['p_np'].tolist() # Assuming tasks start from column 1
    
    dataset = dataset.add_column("labels", labels)
    

    return dataset

In [4]:
from peft import LoraConfig, get_peft_model, PeftModel

def lora_config(r,lora_alpha,dropout):

    lora_config = LoraConfig(
        task_type="SEQ_CLS",  # Sequence classification task
        r=r,  # Rank of LoRA matrices
        lora_alpha=lora_alpha,  # Scaling factor double of rank( from the rule of thumb)
        target_modules='all-linear',
        lora_dropout=dropout  # Dropout rate
        #init_lora_weights="gaussian"
    )

    return lora_config

### Weighted Loss Trainer

In [13]:
import torch

class_weights= [1-(train_dataset['labels'].count(0)/len(train_dataset['labels'])),
                           1-(train_dataset['labels'].count(1)/len(train_dataset['labels']))]

class_weights = torch.from_numpy(np.array(class_weights)).float().to("cuda")

class WeightedLossTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Extract labels
        labels = inputs.get("labels")

        # compute custom loss (suppose one has 2 labels with different weights)
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights)

        # compute loss
        loss = loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

### Focal Loss Trainer

In [5]:
#focal loss computation
import torch.nn.functional as F
import torch

def focal_loss_multiclass(inputs, targets, alpha=1, gamma=2):
    log_prob = F.log_softmax(inputs, dim=-1)
    prob = torch.exp(log_prob)  # Convert log probabilities back to normal probabilities

    targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
    pt = torch.sum(prob * targets_one_hot, dim=-1)  # Get probability of the true class

    focal_loss = -alpha * (1 - pt) ** gamma * torch.sum(log_prob * targets_one_hot, dim=-1)
    
    return focal_loss.mean() 


class FocalLossTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = focal_loss_multiclass(logits, labels)
        
        return (loss, outputs) if return_outputs else loss 

In [6]:
from evaluate import load
import numpy as np
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,matthews_corrcoef

accuracy_metric = load("accuracy")
mcc_metric= load("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    probabilities = softmax(logits, axis=1)[:, 1]  # Get probabilities for class 1
    predictions = np.argmax(logits, axis=1)  # Choose the most likely class
    

    mcc = matthews_corrcoef(labels, predictions)

    return {
        "eval_mcc_metric": mcc,
        "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "AUC-ROC": roc_auc_score(labels, probabilities),  # AUC-ROC requires probabilities
        "Precision": precision_score(labels, predictions),
        "Recall": recall_score(labels, predictions),
        "F1-score": f1_score(labels, predictions)
    } 



### Weighted Loss Trainer

In [7]:

# Initialize W&B with sweep
def run_training():
    run = wandb.init(project="BBBP chemberta Hyperparameter Tuning")
    config = run.config   

    # Define unique save path for each W&B run
    save_dir = f"./models_BBBP_chemberta_77M_MLM_FL/{wandb.run.id}"  # Unique directory for each run
    os.makedirs(save_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
    model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MLM",                                 # Define any other model here
    num_labels=2,
    problem_type="single_label_classification",
    trust_remote_code=True
    )

    # Load data
    train_data, val_data = data_load()
    training_data = data_prep(train_data, tokenizer)
    validation_data = data_prep(val_data, tokenizer)


    # Apply LoRA
    peft_config = lora_config(config.r, config.lora_alpha, config.dropout)
    lora_model = get_peft_model(model, peft_config)
    
    lora_model.print_trainable_parameters()

    # Training arguments
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="epoch",
        learning_rate=config.lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        weight_decay=0.01,
        save_strategy="epoch",  # Save model at each epoch
        logging_dir=f"./logs_BBBP/{wandb.run.id}",
        logging_strategy="steps",
        logging_steps=100,
        report_to="wandb",
        save_total_limit=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_mcc_metric"
    )

    accuracy_metric = load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)[:, 1]  
        predictions = np.argmax(logits, axis=1)  
        mcc = matthews_corrcoef(labels, predictions)

        return {
            "eval_mcc_metric": mcc,
            "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities),
            "Precision": precision_score(labels, predictions),
            "Recall": recall_score(labels, predictions),
            "F1-score": f1_score(labels, predictions)
        }

    # Initialize trainer
    trainer = WeightedLossTrainer(
        model=lora_model,
        args=training_args,
        train_dataset=training_data,
        eval_dataset=validation_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    # Train model
    trainer.train()

    # Save model and tokenizer for this run
    trainer.save_model(save_dir)
    
    print(f"Model saved to {save_dir}")
    
    wandb.finish()


### Focal loss trainer

In [7]:


# Initialize W&B with sweep
def run_training():
    run = wandb.init(project="BBBP focal loss Hyperparameter Tuning")
    config = run.config   

    # Define unique save path for each W&B run
    save_dir = f"./models_BBBP_chemberta_5M_MTR_FL/{wandb.run.id}"  # Unique directory for each run
    os.makedirs(save_dir, exist_ok=True)

    # Load model directly


    # Load model directly


    ## Model 5M MTR Model



    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-5M-MTR")
    model = AutoModelForSequenceClassification.from_pretrained(
        "DeepChem/ChemBERTa-5M-MTR",
        num_labels=2,
        trust_remote_code=True
    )

    # Load data
    train_data, val_data = data_load()
    training_data = data_prep(train_data, tokenizer)
    validation_data = data_prep(val_data, tokenizer)


    # Apply LoRA
    peft_config = lora_config(config.r, config.lora_alpha, config.dropout)
    lora_model = get_peft_model(model, peft_config)
    
    lora_model.print_trainable_parameters()

    # Training arguments
    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="epoch",
        learning_rate=config.lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        weight_decay=0.01,
        save_strategy="epoch",  # Save model at each epoch
        logging_dir=f"./logs_BBBP/{wandb.run.id}",
        logging_strategy="steps",
        logging_steps=100,
        report_to="wandb",
        save_total_limit=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_mcc_metric"
    )

    accuracy_metric = load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)[:, 1]  
        predictions = np.argmax(logits, axis=1)  
        mcc = matthews_corrcoef(labels, predictions)

        return {
            "eval_mcc_metric": mcc,
            "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities),
            "Precision": precision_score(labels, predictions),
            "Recall": recall_score(labels, predictions),
            "F1-score": f1_score(labels, predictions)
        }

    # Initialize trainer
    trainer = FocalLossTrainer(
        model=lora_model,
        args=training_args,
        train_dataset=training_data,
        eval_dataset=validation_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    # Train model
    trainer.train()

    # Save model and tokenizer for this run
    trainer.save_model(save_dir)
    
    print(f"Model saved to {save_dir}")
    
    wandb.finish()


In [8]:
def main():

    sweep_config = {
    "name": "BBBP Chemberta Hyperparameter Tuning",
    "method": "bayes",
    "metric": {
        "goal": "maximize", 
        "name": "eval/mcc_metric"
    },
    "parameters": {
        "lr": {
            "distribution": "uniform",
            "min": 1e-5,  
            "max": 2e-5
        },
        "r": {
            "values": [8,16,32,64, 128]
        },
        "lora_alpha": {
            "values": [8,16,32,64,128]
        },
        "dropout": {
            "values": [0.0,0.1,0.2]
        },
        
        "optimizer": {
            "value": ["adamw"]
        }
    }
}

    sweep_id = wandb.sweep(sweep_config, project="huggingface")
    wandb.agent(sweep_id, function=run_training, count=10)

    api = wandb.Api()
    sweep = api.sweep(f"huggingface/{sweep_id}")
    print(sweep.runs[0].summary_metrics)

    runs_with_rmse = [run for run in sweep.runs if 'eval/mcc_metric' in run.summary_metrics]
    if runs_with_rmse:
        # Sort by rmse in descending order (maximize)
        best_run = sorted(runs_with_rmse, key=lambda run: run.summary_metrics['eval/mcc_metric'])[0]
    else:
        raise ValueError("No runs found with 'eval/mcc_metric' metric.")

    best_hyperparameters = best_run.config
    print(f"Best hyperparameters: {best_hyperparameters}")
    

if __name__ == "__main__":
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    main()

Create sweep with ID: c918zhrd
Sweep URL: https://wandb.ai/harodharsha21-iit-ropar/huggingface/sweeps/c918zhrd


wandb: Agent Starting Run: 4rr2blnx with config:
wandb: 	dropout: 0.1
wandb: 	lora_alpha: 16
wandb: 	lr: 1.9076650745205e-05
wandb: 	optimizer: ['adamw']
wandb: 	r: 64
wandb: Currently logged in as: harodharsha21 (harodharsha21-iit-ropar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-5M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 1,137,922 || all params: 4,639,988 || trainable%: 24.5242


...

/tmp/ipykernel_1816725/2257317317.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` instead.
  trainer = FocalLossTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Mcc Metric,Accuracy,Auc-roc,Precision,Recall,F1-score
1,No log,0.131618,0.227441,0.731707,0.914483,0.731959,0.979310,0.837758
2,0.130000,0.115527,0.296294,0.746341,0.925862,0.740933,0.986207,0.846154
3,0.130000,0.107759,0.332410,0.756098,0.930230,0.751323,0.979310,0.850299
4,0.090500,0.103009,0.385941,0.770732,0.931264,0.763441,0.979310,0.858006
5,0.090500,0.099131,0.466037,0.795122,0.931494,0.784530,0.979310,0.871166
6,0.080000,0.096897,0.510343,0.809756,0.931034,0.797753,0.979310,0.879257
7,0.080000,0.095968,0.510343,0.809756,0.930000,0.797753,0.979310,0.879257
8,0.069000,0.095546,0.566391,0.829268,0.929540,0.816092,0.979310,0.890282
9,0.069000,0.094824,0.593411,0.839024,0.929885,0.825581,0.979310,0.895899
10,0.066100,0.095199,0.604781,0.843902,0.929195,0.834320,0.972414,0.898089


Model saved to ./models_BBBP_chemberta_5M_MTR_FL/kvx5oe5b


eval/AUC-ROC,▁▆▇███▇▇▇▇▇▇▇▇▆▆▆▆
eval/Accuracy,▁▂▂▃▅▆▆▇▇█████████
eval/F1-score,▁▂▂▃▅▆▆▇▇█████████
eval/Precision,▁▂▂▃▄▅▅▆▇█████████
eval/Recall,▆█▆▆▆▆▆▆▆▃▃▃▃▃▃▃▁▁
eval/loss,█▅▃▃▂▁▁▁▁▁▁▁▁▂▂▂▂▂
eval/mcc_metric,▁▂▃▄▅▆▆▇██████████
eval/runtime,▁▃▂█▁▂▄▂▂▃▃▂▂▄▃▅▄▄
eval/samples_per_second,█▆▇▁█▇▅▇▇▆▆▇▇▅▆▄▅▅
eval/steps_per_second,█▆▇▁█▇▅▇▇▆▆▇▇▅▆▄▅▅
train/epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇████


{'_runtime': 51.083988233, '_step': 27, '_timestamp': 1744197660.0279956, '_wandb': {'runtime': 51}, 'eval/AUC-ROC': 0.9271264367816092, 'eval/Accuracy': 0.8439024390243902, 'eval/F1-score': 0.8974358974358975, 'eval/Precision': 0.8383233532934131, 'eval/Recall': 0.9655172413793104, 'eval/loss': 0.10013064742088318, 'eval/mcc_metric': 0.603604942410772, 'eval/runtime': 0.1319, 'eval/samples_per_second': 1554.187, 'eval/steps_per_second': 53.07, 'total_flos': 68890114440720, 'train/epoch': 18, 'train/global_step': 936, 'train/grad_norm': 0.2171177119016647, 'train/learning_rate': 1.8641823066679656e-06, 'train/loss': 0.0594, 'train_loss': 0.07510012744838356, 'train_runtime': 45.5273, 'train_samples_per_second': 720.008, 'train_steps_per_second': 22.843}
Best hyperparameters: {'r': 128, 'lr': 1.1720367087103236e-05, 'bf16': False, 'fp16': False, 'fsdp': [], 'seed': 42, 'tf32': None, 'debug': [], 'optim': 'adamw_torch', 'top_k': 50, 'top_p': 1, 'is_gpu': True, 'prefix': None, 'do_eval': 

Evaluate on Test Dataset

In [9]:
import torch
import numpy as np
import random

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Ensure deterministic behavior in PyTorch computations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


## 77M MTR Model Focal Loss

In [11]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-5M-MTR",
    num_labels=2,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-5M-MTR",trust_remote_code=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-5M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 77M MTR Weighted Loss

In [21]:
## 77M MTR Weighted Loss

In [12]:
import pandas as pd

test_data=pd.read_csv('/home/raghvendra2/Molformer_Finetuning/BBBP_Prediction_Task/bbbp/test.csv')

In [13]:
from datasets import Dataset

smiles_test = test_data['smiles'].tolist()

test_tokenized =tokenizer(smiles_test)

test_dataset = Dataset.from_dict(test_tokenized)

In [14]:
test_labels = test_data['p_np'].tolist() 


test_dataset = test_dataset.add_column("labels", test_labels)

In [15]:
from evaluate import load
import numpy as np
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

accuracy_metric = load("accuracy")

def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probabilities = softmax(logits, axis=1)[:, 1]  # Get probabilities for class 1
        predictions = np.argmax(logits, axis=1)  # Choose the most likely class
        mcc = matthews_corrcoef(labels, predictions)
        
        
        return {
            "eval_mcc_metric": mcc,
            "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
            "AUC-ROC": roc_auc_score(labels, probabilities),  # AUC-ROC requires probabilities
            "Precision": precision_score(labels, predictions),
            "Recall": recall_score(labels, predictions),
            "F1-score": f1_score(labels, predictions)
        }

In [16]:
from transformers import Trainer, TrainingArguments

eval_args = TrainingArguments(
    output_dir="./test_results_clintox_wandb",
    per_device_eval_batch_size=16,
    report_to="none",  # Disable logging to W&B for test

)


In [ ]:
# List all checkpoints inside models directory
import os
from peft import PeftModel

models_dir = "/clintox_models_10MTR"

def find_all_checkpoints(base_dir):
    all_checkpoints = []
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path) and subfolder.startswith("checkpoint-"):
                    if os.path.exists(os.path.join(subfolder_path, "adapter_config.json")):
                        all_checkpoints.append(subfolder_path)
    return all_checkpoints

valid_checkpoints = find_all_checkpoints(models_dir)
print(" Valid nested checkpoints found:", valid_checkpoints)

for checkpoint_path in valid_checkpoints:
    checkpoint_name = os.path.basename(checkpoint_path)
    parent_folder = os.path.basename(os.path.dirname(checkpoint_path))

    print(f"\n Evaluating model: {parent_folder}/{checkpoint_name}")

    adapter_model = PeftModel.from_pretrained(base_model, checkpoint_path)
    adapter_model.eval()

    trainer = FocalLossTrainer(
        model=adapter_model,
        args=eval_args,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )


    test_results = trainer.evaluate()

    auc_score = test_results["eval_AUC-ROC"]
    if auc_score > 0.96:
        print(f"✅ AUC_ROC > 0.90 for {parent_folder}/{checkpoint_name}")
        print(f"📌 Test Results: {test_results}")
    else:
        print(f"❌ Skipping {parent_folder}/{checkpoint_name}")




/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1816725/4183930858.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` inst

🧠 Valid nested checkpoints found: ['./models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-884', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-468', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-728', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-572', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-676', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-832', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-624', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-780', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-416', './models_BBBP_chemberta_5M_MTR_FL/syqmo6uu/checkpoint-520', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkpoint-468', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkpoint-728', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkpoint-572', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkpoint-676', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkpoint-832', './models_BBBP_chemberta_5M_MTR_FL/ftemtdss/checkp

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1816725/4183930858.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` inst

❌ Skipping syqmo6uu/checkpoint-884

🔍 Evaluating model: syqmo6uu/checkpoint-468


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1816725/4183930858.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` inst

...

/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1816725/4183930858.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` inst

❌ Skipping kwtwjebf/checkpoint-416

🔍 Evaluating model: kwtwjebf/checkpoint-260


/storage/qnap_home/raghvendra2/micromamba/envs/Molformer/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.classifier.original_module.dense.lora_A.default.weight', 'base_model.model.classifier.original_module.dense.lora_B.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_A.default.weight', 'base_model.model.classifier.original_module.out_proj.lora_B.default.weight', 'base_model.model.classifier.dense.lora_A.default.weight', 'base_model.model.classifier.dense.lora_B.default.weight', 'base_model.model.classifier.out_proj.lora_A.default.weight', 'base_model.model.classifier.out_proj.lora_B.default.weight']
  warnings.warn(f"Found missing adapter keys while loading the checkpoint: {missing_keys}")
/tmp/ipykernel_1816725/4183930858.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` inst

❌ Skipping kwtwjebf/checkpoint-260

🔍 Evaluating model: kwtwjebf/checkpoint-520


❌ Skipping kwtwjebf/checkpoint-520


### Best results for 77M MTR: p4hh23r5/checkpoint-780  (Focal loss)

trainable params: 395,938 || all params: 3,842,612 || trainable%: 10.3039

{'eval_mcc_metric': 0.7080332487622075, 'eval_loss': 0.06555292755365372, 'eval_model_preparation_time': 0.0049, 'eval_Accuracy': 0.865979381443299, 'eval_AUC-ROC': 0.9654185274247108, 'eval_Precision': 0.8540145985401459, 'eval_Recall': 0.9512195121951219, 'eval_F1-score': 0.9, 'eval_runtime': 0.2057, 'eval_samples_per_second': 943.098, 'eval_steps_per_second': 63.197}

### Best results for 77M MTR: 6mmywjr3/checkpoint-260( weighted loss)

trainable params: 395,938 || all params: 3,842,612 || trainable%: 10.3039

{'eval_mcc_metric': 0.7530924327044072, 'eval_loss': 0.4659155607223511, 'eval_model_preparation_time': 0.004, 'eval_Accuracy': 0.8865979381443299, 'eval_AUC-ROC': 0.9586625443719226, 'eval_Precision': 0.8796992481203008, 'eval_Recall': 0.9512195121951219, 'eval_F1-score': 0.9140625, 'eval_runtime': 0.1762, 'eval_samples_per_second': 1100.732, 'eval_steps_per_second': 73.76}


### Best results for 77M MLM : 3inhdg73/checkpoint-988 (Focal Loss)

trainable params: 643,266 || all params: 4,108,404 || trainable%: 15.6573

📌 Test Results: {'eval_mcc_metric': 0.6713501414672036, 'eval_loss': 0.08056607097387314, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.845360824742268, 'eval_AUC-ROC': 0.9396541852742472, 'eval_Precision': 0.8907563025210085, 'eval_Recall': 0.8617886178861789, 'eval_F1-score': 0.8760330578512396, 'eval_runtime': 0.173, 'eval_samples_per_second': 1121.312, 'eval_steps_per_second': 75.139}

### Best result for 10M MTR: qx65by2n/checkpoint-780

trainable params: 643,266 || all params: 4,108,404 || trainable%: 15.6573

📌 Test Results: {'eval_mcc_metric': 0.7080332487622075, 'eval_loss': 0.07051488757133484, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.865979381443299, 'eval_AUC-ROC': 0.9607236917439598, 'eval_Precision': 0.8540145985401459, 'eval_Recall': 0.9512195121951219, 'eval_F1-score': 0.9, 'eval_runtime': 0.1679, 'eval_samples_per_second': 1155.233, 'eval_steps_per_second': 77.412}

### Best model for 10M MLM:  g0lp7aba/checkpoint-676

trainable params: 1,137,922 || all params: 4,639,988 || trainable%: 24.5242

📌 Test Results: {'eval_mcc_metric': 0.7438155398213232, 'eval_loss': 0.0648377314209938, 'eval_model_preparation_time': 0.0039, 'eval_Accuracy': 0.8814432989690721, 'eval_AUC-ROC': 0.9590060689339288, 'eval_Precision': 0.9032258064516129, 'eval_Recall': 0.9105691056910569, 'eval_F1-score': 0.9068825910931174, 'eval_runtime': 0.1688, 'eval_samples_per_second': 1149.189, 'eval_steps_per_second': 77.008}

### Best Model for 5M MTR: l2kgceha/checkpoint-624

trainable params: 643,266 || all params: 4,108,404 || trainable%: 15.6573


📌 Test Results: {'eval_mcc_metric': 0.6967618094157277, 'eval_loss': 0.0720517635345459, 'eval_model_preparation_time': 0.0038, 'eval_Accuracy': 0.8608247422680413, 'eval_AUC-ROC': 0.9620977899919845, 'eval_Precision': 0.8478260869565217, 'eval_Recall': 0.9512195121951219, 'eval_F1-score': 0.896551724137931, 'eval_runtime': 0.1699, 'eval_samples_per_second': 1141.757, 'eval_steps_per_second': 76.51}





In [36]:



base_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepChem/ChemBERTa-77M-MTR",
    num_labels=2,
    problem_type="single_label_classification",    
    trust_remote_code=True,
    
)

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR",trust_remote_code=True)

# Apply LoRA
peft_config = lora_config(16, 32, 0.1)
lora_model = get_peft_model(base_model, peft_config)

lora_model.print_trainable_parameters()




Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 395,938 || all params: 3,842,612 || trainable%: 10.3039
